# Code for AICHIスキルシェア（jupyter-notebookを使ってみる編）
今まで動かしてきたサンプルから、Watson StudioはWatson APIを動かすためのツールだということがお分かりいただけたかと思います。ですが、Watson StudioのNotebookはjupyer-notebookとして使用することができます。jupyter-notebookは他のプログラミングツールと違い、以下の特徴があります。

- あとから実行結果が確認できる
- 実行手順を確認できる（セルの左端に数字が表示される）
- markdownでメモを残せるので、手順を読みながらプログラムを実行して動作確認出来る。（今回の資料もそうです）

jupyter-notebookは以上の特徴があることから研究でデータ解析をする際によく使われます。というわけでここでは実際にjupyter-notebookを使ってちょっとしたデータを取り扱うことをやっていきます。今回は以前C4Aで紹介されたRESAS APIを使ってデータを取り込んであれこれいじっていきたいと思います。

## RESAS APIキーの登録
RESAS APIを使うためにはAPIの利用者登録をします。未登録の方は[こちら](https://opendata.resas-portal.go.jp/form.html)から利用者登録を行ってください。登録を行ったら、[マイページ](https://opendata.resas-portal.go.jp/mypage.html)からAPIキーをコピーして、下のセルに貼り付けて実行してください。

In [ ]:
API_KEY = "YOUR_API_KEY"

## 都道府県と市区町村コードの一覧
RESASのデータをインポートするには都道府県と市区町村コードが必要です。というわけでまずは都道府県コードを取得していきます。下のセルを実行してください。

In [ ]:
import json
import urllib.request
import pprint
import pandas as pd

api_key = {"X-API-KEY":API_KEY}
url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'

req = urllib.request.Request(url, headers=api_key)
with urllib.request.urlopen(req) as response:
    data = response.read()

d = json.loads(data.decode())
df = pd.io.json.json_normalize(d['result'])
display(df)

これで都道府県の一覧を取得できたと思います。ではこのなかから好きな都道府県を選び、その市区町村の一覧を表示させます。都道府県コードを以下のセルの```prefCode```の隣の値に記入して実行します。（今回は愛知県の市区町村の一覧を出力させます）

In [ ]:
p = {"prefCode":23}

url_base = 'https://opendata.resas-portal.go.jp/api/v1/cities'
url = url_base + "?" + urllib.parse.urlencode(p)

req = urllib.request.Request(url, headers=api_key)

with urllib.request.urlopen(req) as response:
    data = response.read()

d = json.loads(data.decode())

df = pd.io.json.json_normalize(d['result'])
display(df)

## 何かデータを取り出してみる
市区町村コードがわかったところで一人あたりの地方税を取得していきます。下記のセルを実行します。(今回は岡崎市で見ていきます)

In [ ]:
url_base = 'https://opendata.resas-portal.go.jp/api/v1/municipality/taxes/perYear'

p = {'prefCode': 23, 'cityCode': 23202}
url = url_base + '?' + urllib.parse.urlencode(p)

req = urllib.request.Request(url, headers=api_key)

with urllib.request.urlopen(req) as response:
    data = response.read()

d = json.loads(data.decode())

df = pd.io.json.json_normalize(d['result']['data'])
df.columns = ["tax(*1000)", "year"]  # 出力される値に1000をかけた値が一人あたりの地方税になるので、カラムを変える
display(df)

これでデータを整形できたので、今度はこれをグラフに書き起こしてみます。下記のセルを実行しましょう。（エラーもなく、何も表示されない時はもう一回実行してください）

In [ ]:
df.plot(x='year', y='tax(*1000)', color='red')

# 人口ピラミッドを作ってみる
今度はRESUS APIを使って人口ピラミッドを作ってみます。まずは以下のセルを実行し、データをインポートします。（今回は岡崎の人口ピラミッドを作ります）

In [ ]:
url_base = 'https://opendata.resas-portal.go.jp/api/v1/population/composition/pyramid'

p = {'prefCode': 23, 'cityCode': 23202, 'yearLeft':2010, "yearRight":2020}
url = url_base + '?' + urllib.parse.urlencode(p)

req = urllib.request.Request(url, headers=api_key)

with urllib.request.urlopen(req) as response:
    data = response.read()

d = json.loads(data.decode())

print(json.dumps(d, indent=2, ensure_ascii=False))

実行してみると分かりますが、2つの年代を比較できるように結果が返ってきます。今回は2010年と2020年の比較で出します。ではせっかくなので、この2つの年の人口の遷移を比較していきたいと思います。その前に比較しやすくするために表を横並びにします。そのために以下のセルを実行し、関数を定義していきます。

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

それでは表を出力していきましょう。下記のセルを実行します。（今回は左が2010年、右は2020年の人口構成として出力しました）

In [ ]:
dfL = pd.io.json.json_normalize(d['result']['yearLeft']['data'])
dfR = pd.io.json.json_normalize(d["result"]["yearRight"]["data"])
display_side_by_side(dfL, dfR)

これで必要なものが揃ったので実際にグラフを出力していきます。下記のセルを実行します。

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

#日本語フォントの導入  
jp_font_path ='ipaexg00301/ipaexg.ttf'

if not os.path.exists(jp_font_path):
    !wget https://oscdl.ipa.go.jp/IPAexfont/ipaexg00301.zip
    !unzip ipaexg00301.zip
else:
    print('IPA font haｓ been already installed')

fp = FontProperties(fname=jp_font_path, size=10)

man = dfL["man"]
woman = dfL["woman"]

fig, ax = plt.subplots(ncols=2, figsize=(8,6))

# 男性人口
ax[0].barh(dfL["class"], man, color="b", height=0.5, label="man")
ax[0].yaxis.tick_right()
ax[0].set_yticklabels(dfL["class"], FontProperties=fp)
ax[0].set_xlim([20000, 0])
# 女性人口
ax[1].barh(dfL["class"], woman, color="r", height=0.5, label="woman")
ax[1].set_yticklabels([])
ax[1].set_xlim([0, 20000])
ax[1].set_xlabel("人口(人)", FontProperties=fp)

fig.legend(loc="upper right")
plt.suptitle("2010年度", FontProperties=fp)
plt.show()